# 제품 이상여부 판별 프로젝트


## hard voting (csv 파일 이용)

In [1]:
import pandas as pd
import numpy as np
import itertools
import random

# 파일 이름 리스트
file_names1 = [
    "data0825_lgbm.csv",
    "data0825_xgb.csv",
    "data0825_cat.csv",
    "data0825_rf.csv",
    "data0825_et.csv",
    "data0825_ada.csv"
]

file_names2 = [
    "data0825(4)_lgbm.csv",
    "data0825(4)_xgb.csv",
    "data0825(4)_cat.csv",
    "data0825(4)_rf.csv",
    "data0825(4)_et.csv",
    "data0825(4)_ada.csv"
]

# 파일을 무작위로 선택하는 함수
def select_file(name):
    file1 = f"data0825_{name}.csv"
    file2 = f"data0825(4)_{name}.csv"
    return random.choice([file1, file2])

# 파일을 반복할 최대 횟수
max_repeats = 5

# 조합 생성 및 평가
def generate_combinations(file_names, max_repeats):
    combinations = []
    for l in range(max_repeats + 1):
        for x in range(max_repeats + 1):
            for c in range(max_repeats + 1):
                for rf in range(max_repeats + 1):
                    for et in range(max_repeats + 1):
                        for ada in range(max_repeats + 1):
                            current_combination = (
                                [select_file("lgbm")] * l +
                                [select_file("xgb")] * x +
                                [select_file("cat")] * c +
                                [select_file("rf")] * rf +
                                [select_file("et")] * et +
                                [select_file("ada")] * ada
                            )
                            if current_combination:  # 빈 조합은 제외
                                combinations.append(current_combination)
    return combinations

def read_submission_files(file_paths):
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
        predictions.append(preds)
    return predictions

def hard_voting(preds):
    preds = np.array(preds)
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    return final_predictions

def count_single_one_cases(predictions_a, predictions_b):
    count = 0
    for a, b in zip(predictions_a, predictions_b):
        if (a == 1 and b == 0) or (a == 0 and b == 1):
            count += 1
    return count

def evaluate_combination(combination, good_score_predictions):
    file_paths = combination  # 조합된 파일 경로
    predictions = read_submission_files(file_paths)
    final_predictions = hard_voting(predictions)
    single_one_count = count_single_one_cases(final_predictions, good_score_predictions)
    return single_one_count

# good_score.csv 파일 읽기
good_score_df = pd.read_csv("good_score.csv")
good_score_predictions = good_score_df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values

# 조합 생성
all_combinations = generate_combinations(file_names1, max_repeats)

# 최고의 조합 찾기
best_combination = None
best_single_one_count = float('inf')  # 최저 카운트 초기화

for combination in all_combinations:
    single_one_count = evaluate_combination(combination, good_score_predictions)
    
    # '1'이 1개만 있는 경우의 개수가 최소인 조합 찾기
    if single_one_count < best_single_one_count:
        best_single_one_count = single_one_count
        best_combination = combination

print("최고의 조합:", best_combination)
print("'1'이 1개만 있는 경우의 개수:", best_single_one_count)

# 215m 33.2s

최고의 조합: ['data0825(4)_lgbm.csv', 'data0825(4)_lgbm.csv', 'data0825(4)_lgbm.csv', 'data0825(4)_lgbm.csv', 'data0825_xgb.csv', 'data0825_xgb.csv', 'data0825_xgb.csv', 'data0825_xgb.csv', 'data0825_cat.csv', 'data0825_cat.csv', 'data0825_cat.csv', 'data0825_et.csv', 'data0825(4)_ada.csv', 'data0825(4)_ada.csv', 'data0825(4)_ada.csv', 'data0825(4)_ada.csv']
'1'이 1개만 있는 경우의 개수: 146
